In [ ]:
###
# This is used for debugging purposes to see how the image is like after contours are drawn
# Usage: Input the image file name in the area below
###

# Read the image
img = cv2.imread('../' + imageFileSaveArea + '/____')
# Grayscale the read image
im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Apply the bilateral filter onto the grayscale image
bf_image = cv2.bilateralFilter(im_gray, 6, 175, 175)
# Apply the canny edge filter on the bilateral filtered image
ed_image = cv2.Canny(bf_image, lowThresh, high_thresh)

_, contours, _ = cv2.findContours(ed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

contour_list = []
asterisksPresent = 0
for cnts in contours:
    approx = cv2.approxPolyDP(cnts, 0.01*cv2.arcLength(cnts,True),True)
    area = cv2.contourArea(cnts)
    if ((len(approx) > 0) & (area > 30) or ((len(cnts) > 0) & (area > 10))):
        asterisksPresent = asterisksPresent + 1
        contour_list.append(cnts)
        
print 'Asterisks present are: %d' % asterisksPresent        
cv2.drawContours(ed_image, contour_list,  -1, (255,255,0), 2)
plt.imshow(img)
plt.show()

In [ ]:
from PIL import Image
import sys
import pandas as pd
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
import os

In [ ]:
###
# Input the file names
# @input videoFileName - Input video file name
# @input imageFileSaveArea - Folder where extracted images will be saved
# @input frameFileName - CSV file that will contain timestamp data
###

videoFileName = ''
imageFileSaveArea = 'images' + videoFileName
frameFileName = 'frames' + videoFileName + '.csv'

In [ ]:
####
# Find the average threshold value (min threshold and max threshold) across all the images by
# grayscaling each image. The functiosn then return the high_thresh and lowThresh values for each image
# and then they would be divided by the number of images.
####

indir = '../' + imageFileSaveArea
high_thresh = 0
lowThresh = 0
numOfImages = 0

# Recursively go through each picture and get the high and low threshold values of all the images
for root, dirs, filenames in os.walk(indir):
    for f in sorted(filenames):
        read_img = cv2.imread('../' + imageFileSaveArea + '/' + f)
        im_gray = cv2.cvtColor(read_img, cv2.COLOR_BGR2GRAY)
        bi_img = cv2.bilateralFilter(im_gray, 6, 175, 175)
        
        hi_thresh, thresh_im = cv2.threshold(bi_img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        low_thresh = 0.5*hi_thresh
        
        high_thresh += hi_thresh
        lowThresh += low_thresh
        
        numOfImages += 1
        
# Divide the threshold values by the amount of images processed.

print numOfImages
high_thresh = float(high_thresh / numOfImages)
lowThresh = float(lowThresh / numOfImages)

print 'High threshold value is: %d' % high_thresh
print 'Low threshold value is: %d' % lowThresh

In [ ]:
##
# This function checks if the image we are looking at is actually a PIN entry screen at all
#
# @input image - the image we want to check for being a PIN Entry screen
# @returns whether or not we are in a PIN Entry screen

def determine_background(image, coord_topleft, coord_bottomright):
    total_pixels = 0
    num_white_pixels = 0

    # range through our box and check for white pixels
    for x in range(coord_topleft[0], coord_bottomright[0] + 1):
        for y in range(coord_topleft[1], coord_bottomright[1] + 1):
            curr = (x,y)
            r, g, b = image.getpixel(curr)
            if ((r,g,b) == (255,255,255)):
                num_white_pixels += 1
            total_pixels += 1
    if ((float(num_white_pixels) / total_pixels) >= 0.7):
        return False
    else:
        return True


In [ ]:
time_stamp = 0
frame = 1
indir = '../' + imageFileSaveArea
df = pd.DataFrame()

# Has the asterices appear?
firstAsterisk = False
secondAsterisk = False
thirdAsterisk = False
fourthAsterisk = False

for root, dirs, filenames in os.walk(indir):
    for f in sorted(filenames):
        
        # Read Image and grayscale it
        image = cv2.imread('../'+ imageFileSaveArea + '/'+f)
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Preprocessing - Bilaterial Filter smoothes images to reduce noise
        bf_image = cv2.bilateralFilter(img_gray, 6, 175, 175)
        
        # Fills what is in that color range of the filtered image
        edges = cv2.Canny(bf_image,lowThresh,high_thresh)
        # Find contours in the image, reads shapeMask image, only looks for the external hierarchy, and uses simple approximation for contouring
        _, contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Is there an amount of astericies present?
        asterisksPresent = 0

        for cnts in contours:
            approx = cv2.approxPolyDP(cnts, 0.01*cv2.arcLength(cnts,True),True)
            area = cv2.contourArea(cnts)
            if ((len(approx) > 0) & (area > 30) or ((len(cnts) > 0) & (area > 10))):
                asterisksPresent = asterisksPresent + 1
                
        # If the first asterisk is present
        if (asterisksPresent == 1):
            firstAsterisk = True
            secondAsterisk = ''
            thirdAsterisk = ''
            fourthAsterisk = '' 
        # If the second asterisk present
        elif (asterisksPresent == 2):
            secondAsterisk = True
            firstAsterisk = ''
            thirdAsterisk = ''
            fourthAsterisk = '' 
        # If the third asterisk is present
        elif (asterisksPresent == 3):
            thirdAsterisk = True
            firstAsterisk = ''
            secondAsterisk = ''
            fourthAsterisk = ''
        # If the fourth asterisk is present
        elif (asterisksPresent == 4):
            fourthAsterisk = True
            firstAsterisk = ''
            secondAsterisk = ''
            thirdAsterisk = ''
        # If the screen is cleared
        # Don't care about 5th key (?)
        elif (asterisksPresent == 0 or asterisksPresent > 4):
            firstAsterisk = ''
            secondAsterisk = ''
            thirdAsterisk = ''
            fourthAsterisk = '' 

        print 'Asterisks present are: %d in frame: %d' % (asterisksPresent,frame)  
        
        # Checking if background is white, scans a set area.
        log = open(os.path.join(root,f),'r')
        bg = Image.open(log)
        pix = bg.load()
        rgb_im = bg.convert('RGB')
        background_topleft = [3,100]
        background_bottomright = [7,110]
        background = determine_background(bg, background_topleft, background_bottomright)
        
        
        # If the background is white,, output results
        if (background == False):               
            df = df.append(pd.DataFrame({'Background': background, 'First': firstAsterisk, 'Second': secondAsterisk, 'Third': thirdAsterisk, 'Fourth': fourthAsterisk, }, index=[frame]), ignore_index=False)
        # If the background fails to be white, return NaN
        else:
            df = df.append(pd.DataFrame({'Background': background, 'First': 'NaN', 'Second': 'NaN', 'Third': 'NaN', 'Fourth': 'NaN', }, index=[frame]), ignore_index=False)
        
        frame = frame + 1


In [ ]:
#Read the csv file containing the time frame data and output to new csv file
Time = pd.read_csv("../" + frameFileName, 
                  names = ["Time_stamp"])
result = pd.concat([df, Time], axis=1, join='inner')

outputFileForUser = 'results' + videoFileName + '.csv'


result = result.reindex(columns=['Time_stamp','Background', 'First', 'Second', 'Third', 'Fourth']).to_csv('../' + outputFileForUser, index=True)


In [ ]:
# Read that outputted csv file
outputFileForUser = 'results' + videoFileName + '.csv'
nR = pd.read_csv('../' + outputFileForUser, index_col=0)


In [ ]:
# Booleans to find the first keypress and prevent from printing out every single line of the csv file to output

firstKP = False
secondKP = False
thirdKP = False
fourthKP = False
clr = False
start_time = time.time()

# Open and write to a textFile
try: 
    with open('../relevantKeyPresses' + videoFileName + '.txt', 'w') as output:
        # Iterate through every row in the csv file and check specific columns for True values
        for index,row in nR.iterrows():
            # Helps to reset original index
            tempIndex = index

            #Check if the background is representing the text box for the asterisks to be pressed
            if (nR['Background'].iloc[index] == False):

                # Find the first key press
                if (nR['First'].iloc[index] == True and firstKP == False):
                    # Check if first keypress is valid or not by checking if it is the same for future indexes
                    isConsecutive = 0
                    for n in range(0,3):
                        if (nR['First'].iloc[index+n] == True):
                            isConsecutive += 1
                    # If it does prove to be consistent, check if a clear was pressed prior and then print out
                    # the timestamp at which this first keypress occurred and mark the first key as found
                    if (isConsecutive == 3):
                        if (clr == True):
                            clr = False
                        firstKP = True
                        output.write('%s\n' % nR['Time_stamp'].iloc[[index]])
                    # If it fails, reset index value to mitigate what occured before
                    else:
                        index = tempIndex

                # If a user presses clear, account for that
                elif (pd.isnull(nR['First'].iloc[index]) == True and firstKP == True):
                    isConsecutive = 0
                    # Is the clear consistent?
                    for n in range(0,3):
                        if (pd.isnull(nR['First'].iloc[index+n]) == True):
                            isConsecutive += 1
                    # If it is, reset first keypress boolean and make clear boolean true
                    if (isConsecutive == 3):
                        clr = True
                        firstKP = False
                    else:
                        index = tempIndex

                # Find the second key press
                if (nR['Second'].iloc[index] == True and secondKP == False):
                    # Check if the second keypress is valid or not by checking if it is the same for future indexes
                    isConsecutive = 0
                    for n in range (0,3):
                        if (nR['Second'].iloc[index+n] == True):
                            isConsecutive += 1
                    # If it does prove to be consistent, check if a clear was pressed prior and then print out
                    # the timestamp at which this first keypress occurred and mark the second key as found
                    if (isConsecutive == 3):
                        if (clr == True):
                            clr = False
                        secondKP = True
                        output.write('%s\n' % nR['Time_stamp'].iloc[[index]])
                    # If it fails, reset index value to mitigate what occured before
                    else:
                        index = tempIndex

                # Check if a user presses clear after the second keypress was recorded
                elif (pd.isnull(nR['Second'].iloc[index]) == True and secondKP == True):
                    isConsecutive = 0
                    # Is the clear consistent?
                    for n in range(0,3):
                        if (pd.isnull(nR['Second'].iloc[index+n]) == True):
                            isConsecutive += 1
                    #If it is, reset the second keypress boolean and make the clear boolean true
                    if (isConsecutive == 3):
                        clr = True
                        secondKP = False
                    # If it fails, reset index value to mitigate what occured before
                    else:
                        index = tempIndex

                # Find the third keypress
                if (nR['Third'].iloc[index] == True and thirdKP == False):
                    isConsecutive = 0
                    # Check if the third keypress is valid or not by checking if it is the same for future indexes
                    for n in range (0,3):
                        if (nR['Third'].iloc[index+n] == True):
                            isConsecutive += 1
                    # If it does prove to be consistent, check if a clear was pressed prior and then print out
                    # the timestamp at which this first keypress occurred and mark the third key as found
                    if (isConsecutive == 3):
                        if (clr == True):
                            clr = False
                        thirdKP = True
                        output.write('%s\n' % nR['Time_stamp'].iloc[[index]])
                    # If it fails, reset index value to mitigate what occured before
                    else:
                        index = tempIndex
                # Check if a clear has occured after the third keypress
                elif (pd.isnull(nR['Third'].iloc[index]) == True and thirdKP == True):
                    isConsecutive = 0
                     # Is the clear consistent?
                    for n in range (0,3):
                        if (pd.isnull(nR['Fourth'].iloc[index]) == True):
                            isConsecutive += 1
                    #If it is, reset the third keypress boolean and make the clear boolean true
                    if (isConsecutive == 3):
                        clr = True
                        thirdKP = False
                    # If it fails, reset index value to mitigate what occured before
                    else:
                        index = tempIndex

                # Find the fourth keypress
                if (nR['Fourth'].iloc[index] == True and fourthKP == False):
                    isConsecutive = 0
                    for n in range (0,3):
                        if (nR['Fourth'].iloc[index+n] == True):
                            isConsecutive += 1
                    # If it does prove to be consistent, check if a clear was pressed prior and then print out
                    # the timestamp at which this first keypress occurred and mark the fourth key as found
                    if (isConsecutive == 3):
                        if (clr == True):
                            clr = False
                        fourthKP = True
                        output.write('%s\n' % nR['Time_stamp'].iloc[[index]])
                    else:
                        index = tempIndex

                elif (pd.isnull(nR['Fourth'].iloc[index]) == True and fourthKP == True):
                    isConsecutive = 0
                    # Check if the user pressed CLEAR after the fourth keystroke
                    if (nR['Third'].iloc[index] == True):
                        for n in range (0,3):
                            if (nR['Third'].iloc[index+n] == True):
                                isConsecutive += 1
                        # If it fails, reset index value to mitigate what occured before
                        if (isConsecutive == 3):
                            fourthKP = False
                            clr = True
                        else:
                            index = tempIndex
                    # An enter has been pressed and the screen has been cleared
                    else:
                        firstKP = False
                        secondKP = False
                        thirdKP = False
                        fourthKP = False
                        output.write('%s\n' % nR['Time_stamp'].iloc[[index]])
                        output.write('------------------------\n')
            # In the case at which the user presses enter and causes the pin to show up again, or the system shows the
            # break immediately after typing the correct PIN.
            else:
                # If the user pressed enter after the first keypress
                if (firstKP == True and pd.isnull(nR['First'].iloc[index]) == True and pd.isnull(nR['Fourth'].iloc[index]) == True
                      and pd.isnull(nR['Second'].iloc[index]) == True and pd.isnull(nR['Third'].iloc[index]) == True):
                    firstKP = False
                    secondKP = False
                    thirdKP = False
                    fourthKP = False
                    output.write('%s\n' % nR['Time_stamp'].iloc[[tempIndex]])
                    output.write('------------------------\n')
                # If the user pressed enter after the second keypress
                elif (secondKP == True and pd.isnull(nR['Second'].iloc[index]) == True and pd.isnull(nR['Fourth'].iloc[index]) == True
                 and pd.isnull(nR['First'].iloc[index]) == True and pd.isnull(nR['Third'].iloc[index]) == True):
                    firstKP = False
                    secondKP = False
                    thirdKP = False
                    fourthKP = False
                    output.write('%s\n' % nR['Time_stamp'].iloc[[tempIndex]])
                    output.write('------------------------\n')
                # If the user pressed enter after the third keypress
                elif (thirdKP == True and pd.isnull(nR['Third'].iloc[index]) == True and pd.isnull(nR['Fourth'].iloc[index]) == True 
                	and pd.isnull(nR['First'].iloc[index]) == True and pd.isnull(nR['Second'].iloc[index]) == True ):
                    firstKP = False
                    secondKP = False
                    thirdKP = False
                    fourthKP = False
                    output.write('%s\n' % nR['Time_stamp'].iloc[[tempIndex]])
                    output.write('------------------------\n')
                # If the user pressed enter after the fourth keypress, this also accounts the error at which the user
                elif (fourthKP == True and pd.isnull(nR['Fourth'].iloc[index]) == True and pd.isnull(nR['First'].iloc[index]) == True 
                      and pd.isnull(nR['Second'].iloc[index]) == True and pd.isnull(nR['Third'].iloc[index]) == True):
                    firstKP = False
                    secondKP = False
                    thirdKP = False
                    fourthKP = False
                    output.write('%s\n' % nR['Time_stamp'].iloc[[tempIndex-1]])
                    output.write('------------------------\n')
except:
    print 'EOF'

# Clean text file up and output to a new textfile
with open('../relevantKeyPresses' + videoFileName + '.txt', 'r') as input:
    with open('../cleanedRelevantKeyPresses' + videoFileName + '.txt','wb') as output: 
        for line in input:
            if (line!="Name: Time_stamp, dtype: float64"+"\n"):
                output.write(line)
print 'Complete'
print "My program took", time.time() - start_time, "to run"